In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'pii-detection-removal-from-educational-data:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F66653%2F7500999%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240303%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240303T201340Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D16e667fb2f7e29d3a5c895a1e4dcc008960d5a7e26e7e2ee76eff4cc0f3fabe1c964033417604c6749798d7181e4acc448459c9ef0baec3aa5dca302f95123e22588e90be6212264537d61846c0af308bd58aeda828b11764cb370517d76cba76894be4f400c7c9236c80826c900a6fa9904d02b030b611b7a1dba4147fcead7e39e9afdce919195984dce18557beef957880b4d3587c8e06d0930c10406806588f1bd10ce621d10c2480660dd1c211b8eaf0722d9528cf2021807a85b4c3f0705299fe3daa5fbd389de5a0fc4a20b9878d78609a9a148fa8ab0438c6488be54795232332c260b5eb6458b81a135fe07133ff544e4c47a805589f9633c16731e,kerasv3-lib-ds:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4308295%2F7526248%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240303%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240303T201340Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D557081090a50b77f3ef1d78535d51a3860dab94da40d7edb5898b9762a128bbbebd8195beaa609deecb130ba1c7f63ee2ca198c57e07013fbe19f49ce8215641aed289306400756989ed1c50b547e80aaf86cec582699e88d8dbd079a12278e2ebb3bcc0c33d9c1b23a8e3b7063e664cbf0996098861665b4d800cd852057a5dc57def3efa5b6a90b9fbac81139a2a4cf8953ce9c1224b80dc5f3d593b17dc6983db02e8b3a53c2ad1583dcf4dab4250fdcd020edfb30d69199eb24367c385b4d897828c9f89f8f070d42ab96d3508bce834159802500b288dd8d100dc5b524bf7f6a36f6bcce3ae301c30f680e03eeeb72871d1305fe8b866d705b79118d77b,pii-data-detection-ckpt-ds:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4442914%2F7626226%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240303%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240303T201340Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D9e5db3a2cc5bdab35fc722bc570039e8dd6569a8d4e688347905a4059d1f806cbf5789fed470ad024f8b869b6e72a3b0b568e72ce56d18859af53b70b2b0f9b4d278c25982ee75572ab3e9372a224b70b3d19c92dec38e47bb2c06ff84e7186ded00562c587a0547d8c3b65dcb8b4fbb3621e55cd5cd88c59c4f2fe27d5d9c49d530bbd1dd33d09f7758479c2780d93fe9e51b8dd485af968a75dc9b720cd08dad7ab760db1d541f363e1180786351845396afea804bd5626b2f7c73492388a761e8b73ad7fe8d6f1e40e689998f730ce342299c770f4b132f5c129bb82af2e895a67f91573346503c0cca5d3cdfbe6c196a109355465981c6ffe10ec8b28cf7,deberta_v3/keras/deberta_v3_small_en/2:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F4685%2F6064%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240303%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240303T201340Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D52dec85a46b40394886501c9cc7afef653afc1f0d0106601f5f6a2e0b3428c45c33ba67910130def60575aac894d03e952ae8b58d30fa7052e4d513d7f0bc12e90f093cf9f1dc2d15b202996db4e5ad3c7760d90de1cb51d6d764b7a1693559bb06a189a4244abde9642187a7692ea3b48c0c8b9885f877ac4bb24b82ef1b7d01b4e552d2d3845498ac5337498c097f283ad4c900a9acb8ce6197d0e3e1e616831ba0aec92fc4c9a57ede87b3fbfe597f9d611a195a581794bd4c39a10b2983ca3aaeafde94a83e18cda29c79745de3f4fc711751274529e7129dbaeacb5f0ea62b1fb7ad187aecdad9feda6969f4b063e3854fe047e13b88a0cafaa63c694b3,deberta_v3/keras/deberta_v3_base_en/2:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F4686%2F6065%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240303%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240303T201340Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D9d0205744fb48f58d75ea811853cf8a25e760def47c8278123d0103e7469ea721c65eafe711d1512c92206d59fae4aee5ec5078b17c0c3bcf52cd6d184782fcca3fd984b3dce4431ecf3a894a520c8d4c97070d215938c0992b8dd9ac9d7a27e985e98e4f501c6aca8df71f644738ab48d90977f3b082d53e6a047a3472517cc04be2fc60dc29b3b97ee8b6a210a62e1189f0610edcbc0a4e4861ca6ab917f8b91552828a80f712f006deb0f34b58f16987ef615167573a81a5feda7af1c6ca5a951e3485e96928ebc05f738f005207d3628d0b101071a8c8298c12b5fa136b1f4e337c45aa8493d0e9f57d96b3ff7e6ebc461f452464f8857c04f206a53d9e8,distil_bert/keras/distil_bert_base_en_uncased/2:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F4689%2F6068%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240303%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240303T201340Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D55fa4a14e4b53d99bd88dfcb0a4ed2737859cb068c31db9a44d7614a74939381ab5ad9b4b012f39e79870c5699c5bbef7ae2b71e725c2282e092ff421a227eb643b8391ea4b82657b4c1242d0b9757188dd657fa87d953b886b7e9cd709b506bc0e112f03acd1e8773c44a7442e17db71aa65a059a717c74f1be105973808b648e79d4360a3f6237c6a6858f14f873c2d0e5c3eca4c22d9b880562bcd20489ff633fdd5737a6672ac0416073160a4beca41d6606d73d71ec6bf9e51b0c156eba13a432f631c91100f5bfe78412c0a913db6100d09f80a2b3d7529468cdf6298428223a54523703ebd56342f5a5e444b1d0ff0da7c09f346b66ce56d86e0aceb1,distil_bert/keras/distil_bert_base_en/2:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F4690%2F6069%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240303%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240303T201340Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D88f0c2636f6e80aa74ea9ed3365c379f73d6deca1237b2c8bf56f3d9d6e9b8700a7e26a14ff79a65cbf963d26062e336ab279275adaf6d66a97e98746e3b76aea497fb37c264692818cc1c31aeed2f08177f10aab6e613673acb4bcbbfb80a2ff5044596e01ba34ab6c9fe2f46c4a5e8179b49f062c80fd1165d35c002978c97231a6b7f0712bee680b386bce4da65774e742c510aa8f7e76887bed4116c58726e28e6cee12a8e5859cbe14610761a78de63bde7242651d575d63398d30c4409de7b040bd86d95bedccb86931a2129dcefa64a9156001c558028be076ea7f363613aab3abcd2a29b1427f7049d512757d63aa627127a59bb3cb6115ca498f235'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


<center><img src="https://keras.io/img/logo-small.png" alt="Keras logo" width="100"><br/>
This starter notebook is provided by the Keras team.</center>

# PII Data Detection with [KerasNLP](https://github.com/keras-team/keras-nlp) and [Keras](https://github.com/keras-team/keras)

> The objective of this competition is to detect and remove personally identifiable information (PII) from student writing.

<div align="center">
    <img src="https://i.ibb.co/3stPB0t/pii-data-detection.jpg" alt="PII Data Detection">
</div>

The task of this competition falls under **Token Classification** (not Text Classification!), sometimes known as **Named Entity Recognition (NER)**. This notebook guides you through performing this task from scratch for the competition. Implementing from scratch is a unique feature of this notebook, as most public notebooks use **HuggingFace** to handle modeling and data processing, which performs many tasks under the hood. One may have to look deeper into the repository to understand what is happening inside. In contrast, this notebook goes step by step, showing you exactly how Token Classification works. A cherry on top: this notebook leverages **Mixed Precision** and **Distributed (multi-GPU)** Training/Inference to turbocharge performance!

<u>Fun fact</u>: This notebook is backend-agnostic, supporting TensorFlow, PyTorch, and JAX. Utilizing KerasNLP and Keras allows us to choose our preferred backend. Explore more details on [Keras](https://keras.io/keras_3/).

In this notebook, you will learn how to:

- Design a data pipeline for token classification.
- Create a model for token classification with KerasNLP.
- Load the data efficiently using [`tf.data`](https://www.tensorflow.org/guide/data).
- Perform Mixed Precision and Distributed Training/Inference with Keras 3.
- Make submission on test data.

**Note**: For a more in-depth understanding of KerasNLP, refer to the [KerasNLP guides](https://keras.io/keras_nlp/).


## 🛠 | Install Libraries  

Since internet access is **disabled** during inference, we cannot install libraries in the usual `!pip install <lib_name>` manner. Instead, we need to install libraries from local files. In the following cell, we will install libraries from our local files. The installation code stays very similar - we just use the `filepath` instead of the `filename` of the library. So now the code is `!pip install <local_filepath>`.

> The `filepath` of these local libraries look quite complicated, but don't be intimidated! Also `--no-deps` argument ensures that we are not installing any additional libraries.

In [ ]:
!pip install -q /kaggle/input/kerasv3-lib-ds/tensorflow-2.15.0.post1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl --no-deps
!pip install -q /kaggle/input/kerasv3-lib-ds/keras-3.0.4-py3-none-any.whl --no-deps

# 📚 | Import Libraries

In [ ]:
import os
os.environ["KERAS_BACKEND"] = "jax" # # you can also use tensorflow or torch

import keras
import keras_nlp
from keras import ops
import tensorflow as tf

import json
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

import plotly.graph_objs as go
import plotly.express as px

## Library Versions

In [ ]:
print("TensorFlow:", tf.__version__)
print("Keras:", keras.__version__)
print("KerasNLP:", keras_nlp.__version__)

# ⚙️ | Configuration

In [ ]:
class CFG:
    seed = 42
    preset = "deberta_v3_small_en" # name of pretrained backbone
    train_seq_len = 1024 # max size of input sequence for training
    train_batch_size = 2 * 8 # size of the input batch in training, x 2 as two GPUs
    infer_seq_len = 2000 # max size of input sequence for inference
    infer_batch_size = 2 * 2 # size of the input batch in inference, x 2 as two GPUs
    epochs = 6 # number of epochs to train
    lr_mode = "exp" # lr scheduler mode from one of "cos", "step", "exp"

    labels = ["B-EMAIL", "B-ID_NUM", "B-NAME_STUDENT", "B-PHONE_NUM",
              "B-STREET_ADDRESS", "B-URL_PERSONAL", "B-USERNAME",
              "I-ID_NUM", "I-NAME_STUDENT", "I-PHONE_NUM",
              "I-STREET_ADDRESS","I-URL_PERSONAL","O"]
    id2label = dict(enumerate(labels)) # integer label to BIO format label mapping
    label2id = {v:k for k,v in id2label.items()} # BIO format label to integer label mapping
    num_labels = len(labels) # number of PII (NER) tags

    train = True # whether to train or use already trained ckpt

# ♻️ | Reproducibility
Sets value for random seed to produce similar result in each run.

In [ ]:
keras.utils.set_random_seed(CFG.seed)

# 🚀 | Distributed Training / Inference

In this notebook, we will also use the `Data Parallel` strategy for **Distributed Training/Inference**. This means that the model weights will be replicated across all devices, and each device will process a portion of the input data.

> **Note**: Currently, `DataParallel` is implemented on the JAX backend, so for TensorFlow and PyTorch backends, we can only use a single GPU.

In [ ]:
# Get devices default "gpu" or "tpu"
devices = keras.distribution.list_devices()
print("Device:", devices)

if len(devices) > 1:
    # Data parallelism
    data_parallel = keras.distribution.DataParallel(devices=devices)

    # Set the global distribution.
    keras.distribution.set_distribution(data_parallel)

# 🧮 | Mixed Precision

To enable larger batch sizes and faster training, we'll utilize `mixed_precision` in this notebook. In Keras, this can be achieved with just **one line of code**, as shown below.

In [ ]:
keras.mixed_precision.set_global_policy("mixed_float16")

# 📁 | Dataset Path

In [ ]:
BASE_PATH = "/kaggle/input/pii-detection-removal-from-educational-data"

# 📖 | Meta Data

The competition dataset contains ~$22,000$ student essays where $70\%$ essays are reserved for **testing**, leaving $30\%$ for **training** and **validation**.

Sure, here's the modified markdown with an example of the BIO format label:

**Data Overview:**

* All essays were written in response to the **same prompt**, applying course material to a real-world problem.
* The dataset includes **7 types of PII**: `NAME_STUDENT`, `EMAIL`, `USERNAME`, `ID_NUM`, `PHONE_NUM`, `URL_PERSONAL`, `STREET_ADDRESS`.
* Labels are given in **BIO (Beginning, Inner, Outer)** format.

**Example of BIO format label:**

Let's consider a sentence: `"The email address of Michael jordan is mjordan@nba.com"`. In BIO format, the labels for the personally identifiable information (PII) would be annotated as follows:

| **Word** | The | email | address | of | Michael | Jordan | is | mjordan@nba.com |
|----------|-----|-------|---------|----|---------|--------|----|----------------|
| **Label** | O   | O     | O       | O  | B-NAME_STUDENT | I-NAME_STUDENT | O  | B-EMAIL        |

In the example above, `B-` indicates the beginning of an PII, `I-` indicates an inner part of a multi-token PII, and `O` indicates tokens that do not belong to any PII.

**Data Format:**

* The train/test data is stored in `{test|train}.json` files.
* Each json file has:
    * `document`: unique ID (integer)
    * `full_text`: essay content (string)
    * `tokens`: individual words in the essay (list of strings)
    * `labels` (training data only): BIO labels for each token (list of strings)

In [ ]:
# Train-Valid data
data = json.load(open(f"{BASE_PATH}/train.json"))

# Initialize empty arrays
words = np.empty(len(data), dtype=object)
labels = np.empty(len(data), dtype=object)

# Fill the arrays
for i, x in tqdm(enumerate(data), total=len(data)):
    words[i] = np.array(x["tokens"])
    labels[i] = np.array([CFG.label2id[label] for label in x["labels"]])

# 📊 | Exploratory Data Analysis

From the following label distribution plot, it is evident that there is a significant **class imbalance** between PII tags. This could be a key area for improvement where **external datasets** and **augmentations** could play a pivotal role.

In [ ]:
# Get unique labels and their frequency
all_labels = np.array([x for label in labels for x in label])
unique_labels, label_counts = np.unique(all_labels, return_counts=True)

# Plotting
fig = go.Figure(data=go.Bar(x=CFG.labels, y=label_counts))
fig.update_layout(
    title="Label Distribution",
    xaxis_title="Labels",
    yaxis_title="Count",
    yaxis_type="log",
)

fig.update_traces(text=label_counts, textposition="outside")
fig.show()


## 🔪 | Data Split

In the following code snippet, we will split the dataset into training and testing subsets using an `80%-20%` ratio.

In [ ]:
# Splitting the data into training and testing sets
train_words, valid_words, train_labels, valid_labels = train_test_split(
    words, labels, test_size=0.2, random_state=CFG.seed
)

# 🍽️ | Pre-Processing

Initially, raw text data is quite complex and challenging for modeling due to its high dimensionality. We simplify this complexity by converting text into words then more manageable set of tokens using `tokenizers`. For example, transforming the sentence `"The quick brown fox"` into tokens like `["the", "qu", "##ick", "br", "##own", "fox"]` helps us break down the text effectively. Then, since models can't directly process strings, they are converted into integers, like `[10, 23, 40, 51, 90, 84]`. Additionally, many models require special tokens and additional tensors to understand input better. A `preprocessing` layer helps with this by adding these special tokens, which aid in separating input and identifying padding, among other tasks.

You can explore the following pages to access the available preprocessing and tokenizer layers in **KerasNLP**:
- [Tokenizers](https://keras.io/api/keras_nlp/tokenizers/)
- [Preprocessing](https://keras.io/api/keras_nlp/preprocessing_layers/)

In [ ]:
# To convert string input or list of strings input to numerical tokens
tokenizer = keras_nlp.models.DebertaV3Tokenizer.from_preset(
    CFG.preset,
)

# Preprocessing layer to add spetical tokens: [CLS], [SEP], [PAD]
packer = keras_nlp.layers.MultiSegmentPacker(
    start_value=tokenizer.cls_token_id,
    end_value=tokenizer.sep_token_id,
    sequence_length=10,
)

## Tokenizer in Action

The following code shows the effects of `DebertaV3Tokenizer`. We can see that the word `["reflexion"]` has been divided into `["▁reflex", "ion"]` tokens. Therefore, for similar cases, it's necessary to align labels of tokens to labels of words.

In [ ]:
sample_words = words[0][:5]
sample_tokens_int = [
    token.tolist() for word in sample_words for token in tokenizer(word)
]
sample_tokens_str = [tokenizer.id_to_token(token) for token in sample_tokens_int]

print("words        :", sample_words.tolist())
print("tokens (str) :", sample_tokens_str)
print("tokens (int) :", sample_tokens_int)


## Preprocessor in Action

Even though we converted string inputs to integer tokens with Tokenizer, we are not done yet. We need to add special tokens like `[CLS]`, `[SEP]`, `[PAD]`. This is wehere `Preprocessing` layer comes into the picture. In this notebook, we will use `MultiSegmentPacker` layer. Let's see it action.

In [ ]:
padded_sample_tokens_int = packer(np.array(sample_tokens_int))[0].tolist()
padded_sample_tokens_str = [
    tokenizer.id_to_token(token) for token in padded_sample_tokens_int
]

print("tokens (str)        :", sample_tokens_str)
print("padded tokens (str) :", padded_sample_tokens_str, "\n")

print("tokens (int)        :", sample_tokens_int)
print("padded tokens (int) :", padded_sample_tokens_int)


# 🥣 | Data Processing

One of the key factors that sets Token Classification apart from Text Classification is the data processing part. Unlike text classification, where we simply send our tokenized text to the model, in token classification, we have to apply more processing before sending it to the model. For example, when the `tokenizer` creates multiple tokens for single word or the `processing` layer adds special tokens `[CLS]`, `[SEP]`, and `[PAD]`, they create a mismatch between the input and labels. Thus, a single word corresponding to a single label may now be split into two tokens. We need to realign the tokens labels with word labels by:

- Mapping tokens label to their corresponding word label using `token_ids`.
- Assigning the label `-100` to special tokens `[CLS]`, `[SEP]` and `[PAD]` to disregard them in the `CrossEntropy` loss calculation.
- Labeling only the first token of each word and assigning `-100` to other tokens belonging to the same word.

Specifically, the following cell contains the following functions:
- `process_data()` - prepares input, label, and token_ids
    - `get_tokens()` - creates input tokens ans padding masks from string words
    - `get_token_ids()` - generates token ids for aligning tokens and labels
    - `get_token_labels()` - realigns token labels and adds padding (`-100`) to match input
    - `process_token_ids()` - adds padding (`-1`) to token ids to match input

In [ ]:
def get_tokens(words, seq_len, packer):
    # Tokenize input
    token_words = tf.expand_dims(
        tokenizer(words), axis=-1
    )  # ex: (words) ["It's", "a", "cat"] ->  (token_words) [[1, 2], [3], [4]]
    tokens = tf.reshape(
        token_words, [-1]
    )  # ex: (token_words) [[1, 2], [3], [4]] -> (tokens) [1, 2, 3, 4]
    # Pad tokens
    tokens = packer(tokens)[0][:seq_len]
    inputs = {"token_ids": tokens, "padding_mask": tokens != 0}
    return inputs, tokens, token_words


def get_token_ids(token_words):
    # Get word indices
    word_ids = tf.range(tf.shape(token_words)[0])
    # Get size of each word
    word_size = tf.reshape(tf.map_fn(lambda word: tf.shape(word)[0:1], token_words), [-1])
    # Repeat word_id with size of word to get token_id
    token_ids = tf.repeat(word_ids, word_size)
    return token_ids


def get_token_labels(word_labels, token_ids, seq_len):
    # Create token_labels from word_labels ->  alignment
    token_labels = tf.gather(word_labels, token_ids)
    # Only label the first token of a given word and assign -100 to others
    mask = tf.concat([[True], token_ids[1:] != token_ids[:-1]], axis=0)
    token_labels = tf.where(mask, token_labels, -100)
    # Truncate to max sequence length
    token_labels = token_labels[: seq_len - 2]  # -2 for special tokens ([CLS], [SEP])
    # Pad token_labels to align with tokens (use -100 to pad for loss/metric ignore)
    pad_start = 1  # for [CLS] token
    pad_end = seq_len - tf.shape(token_labels)[0] - 1  # for [SEP] and [PAD] tokens
    token_labels = tf.pad(token_labels, [[pad_start, pad_end]], constant_values=-100)
    return token_labels


def process_token_ids(token_ids, seq_len):
    # Truncate to max sequence length
    token_ids = token_ids[: seq_len - 2]  # -2 for special tokens ([CLS], [SEP])
    # Pad token_ids to align with tokens (use -1 to pad for later identification)
    pad_start = 1  # [CLS] token
    pad_end = seq_len - tf.shape(token_ids)[0] - 1  # [SEP] and [PAD] tokens
    token_ids = tf.pad(token_ids, [[pad_start, pad_end]], constant_values=-1)
    return token_ids


def process_data(seq_len=720, has_label=True, return_ids=False):
    # To add spetical tokens: [CLS], [SEP], [PAD]
    packer = keras_nlp.layers.MultiSegmentPacker(
        start_value=tokenizer.cls_token_id,
        end_value=tokenizer.sep_token_id,
        sequence_length=seq_len,
    )

    def process(x):
        # Generate inputs from tokens
        inputs, tokens, words_int = get_tokens(x["words"], seq_len, packer)
        # Generate token_ids for maping tokens to words
        token_ids = get_token_ids(words_int)
        if has_label:
            # Generate token_labels from word_labels
            token_labels = get_token_labels(x["labels"], token_ids, seq_len)
            return inputs, token_labels
        elif return_ids:
            # Pad token_ids to align with tokens
            token_ids = process_token_ids(token_ids, seq_len)
            return token_ids
        else:
            return inputs

    return process

# 🍚 | Dataloader

The code below sets up a data flow pipeline using `tf.data.Dataset` for data processing. Notable aspects of `tf.data` include its ability to simplify pipeline construction and represent components in sequences. To learn more about `tf.data`, refer to this [documentation](https://www.tensorflow.org/guide/data).

> **Note**: We have used `ragged` tensor as each row has text with different sizes.

In [ ]:
def build_dataset(words, labels=None, return_ids=False, batch_size=4,
                  seq_len=512, shuffle=False, cache=True, drop_remainder=True):
    AUTO = tf.data.AUTOTUNE

    slices = {"words": tf.ragged.constant(words)}
    if labels is not None:
        slices.update({"labels": tf.ragged.constant(labels)})

    ds = tf.data.Dataset.from_tensor_slices(slices)
    ds = ds.map(process_data(seq_len=seq_len,
                             has_label=labels is not None,
                             return_ids=return_ids), num_parallel_calls=AUTO) # apply processing
    ds = ds.cache() if cache else ds  # cache dataset
    if shuffle: # shuffle dataset
        ds = ds.shuffle(1024, seed=CFG.seed)
        opt = tf.data.Options()
        opt.experimental_deterministic = False
        ds = ds.with_options(opt)
    ds = ds.batch(batch_size, drop_remainder=drop_remainder)  # batch dataset
    ds = ds.prefetch(AUTO)  # prefetch next batch
    return ds

## Build Train & Valid Dataloader

In the following code, we'll create **train** and **valid** data loaders.

In [ ]:
train_ds = build_dataset(train_words, train_labels,  batch_size=CFG.train_batch_size,
                         seq_len=CFG.train_seq_len, shuffle=True)

valid_ds = build_dataset(valid_words, valid_labels, batch_size=CFG.train_batch_size,
                         seq_len=CFG.train_seq_len, shuffle=False)

## Dataset Check

Let's check a batch of samples and their associated labels from the dataset.

In [ ]:
inp, tar = next(iter(valid_ds))
print("# Input:\n",inp); print("\n# Labels:\n",tar)

# 🔍 | Loss & Metric

## Loss: CrossEntropy

To optimize our model we will use `CrossEntropy` loss, also known as log loss. It is defined as:

$$
\text{CrossEntropy} = - \frac{1}{N} \sum_{i=1}^{N} \left[ y_i \log(\hat{y}_i) + (1 - y_i) \log(1 - \hat{y}_i) \right]
$$

Where:
- $N$ is the number of samples.
- $y_i$ is the true label of the $i^{th}$ sample.
- $\hat{y}_i$ is the predicted probability of the $i^{th}$ sample being in the positive class.

> **Note**: We will not compute loss for `ignore_class` which indicates special tokens (`[CLS]`, `[SEP]`, `[PAD]`) or intermediate token of a word.

In [ ]:
class CrossEntropy(keras.losses.SparseCategoricalCrossentropy):
    def __init__(self, ignore_class=-100, reduction=None, **args):
        super().__init__(reduction=reduction, **args)
        self.ignore_class = ignore_class

    def call(self, y_true, y_pred):
        y_true = ops.reshape(y_true, [-1])
        y_pred = ops.reshape(y_pred, [-1, CFG.num_labels])
        loss = super().call(y_true, y_pred)
        if self.ignore_class is not None:
            valid_mask = ops.not_equal(
                y_true, ops.cast(self.ignore_class, y_pred.dtype)
            )
            loss = ops.where(valid_mask, loss, 0.0)
            loss = ops.sum(loss)
            loss /= ops.maximum(ops.sum(ops.cast(valid_mask, loss.dtype)), 1)
        else:
            loss = ops.mean(loss)
        return loss


## Metric: FBetaScore ($\beta = 5$)

The competition metric is $F^\beta$, which combines precision and recall, weighted by a parameter $\beta = 5$. It is defined as:

$$
\text{FBetaScore} = (1 + \beta^2) \times \frac{\text{Precision} \times \text{Recall}}{\beta^2 \times \text{Precision} + \text{Recall}}
$$

Where:
- Precision $= \frac{\text{True Positives}}{\text{True Positives} + \text{False Positives}}$
- Recall $= \frac{\text{True Positives}}{\text{True Positives} + \text{False Negatives}}$
- $\beta$ controls the weighting between precision and recall. As in this competition, $\beta = 5$, it means more weight is given to recall. In other words, **metric will penalize more, if a positive token is classified as negative**.

> **Note${}^1$**: The competition will use `micro` averaging for the `FBetaScore`, considering total counts across all classes, which is influenced by class imbalances. The `macro` averaging treats each class equally, regardless of frequency. Organizers may want models that perform well on predicting more common PII tags.

> **Note${}^2$**: We will not compute the metric for `ignore_classes`, which indicates special tokens (`[CLS]`, `[SEP]`, `[PAD]`) or non-start tokens of a word or `O` (Outer) labels.

In [ ]:
class FBetaScore(keras.metrics.FBetaScore):
    def __init__(self, ignore_classes=[-100, 12], average="micro", beta=5.0,
                 name="f5_score", **args):
        super().__init__(beta=beta, average=average, name=name, **args)
        self.ignore_classes = ignore_classes or []

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = ops.convert_to_tensor(y_true, dtype=self.dtype)
        y_pred = ops.convert_to_tensor(y_pred, dtype=self.dtype)

        y_true = ops.reshape(y_true, [-1])
        y_pred = ops.reshape(y_pred, [-1, CFG.num_labels])

        valid_mask = ops.ones_like(y_true, dtype=self.dtype)
        if self.ignore_classes:
            for ignore_class in self.ignore_classes:
                valid_mask &= ops.not_equal(y_true, ops.cast(ignore_class, y_pred.dtype))
        valid_mask = ops.expand_dims(valid_mask, axis=-1)

        y_true = ops.one_hot(y_true, CFG.num_labels)

        if not self._built:
            self._build(y_true.shape, y_pred.shape)

        threshold = ops.max(y_pred, axis=-1, keepdims=True)
        y_pred = ops.logical_and(
            y_pred >= threshold, ops.abs(y_pred) > 1e-9
        )

        y_pred = ops.cast(y_pred, dtype=self.dtype)
        y_true = ops.cast(y_true, dtype=self.dtype)

        tp = ops.sum(y_pred * y_true * valid_mask, self.axis)
        fp = ops.sum(y_pred * (1 - y_true) * valid_mask, self.axis)
        fn = ops.sum((1 - y_pred) * y_true * valid_mask, self.axis)

        self.true_positives.assign_add(tp)
        self.false_positives.assign_add(fp)
        self.false_negatives.assign_add(fn)

# 🤖 | Modeling

In this notebook, we will use the `DebertaV3` backbone from KerasNLP's pretrained models to extract features of tokens and employ `Dense` layers for token-level classification. Unlike Text Classification, transformer outputs are not pooled; instead, a `Dense` layer is applied to the outputs to obtain predictions.

To clarify, the output of the transformer model is a 3D tensor of shape $(batch\_size, seq\_len, feat\_dim)$, where only the $feat\_dim$ is changed, while the others remain the same. Subsequently, the `Dense` (or `Linear`) layer maps the `feat_dim` to `num_labels` and then applies a `softmax` activation to get the final prediction.

To explore other backbones, simply modify the `preset` in the `CFG` (config). A list of available pretrained backbones can be found on the [KerasNLP website](https://keras.io/api/keras_nlp/models/).

> **Note:** The output `dtype` of the final activation is manually set to `float32` to facilitate `mixed_precision`.

<u>Food for thought</u>:
1. Some may wonder why the input to the `Dense` layer is 3D `(batch_size, d0, d1)` instead of the traditional 2D `(batch_size, d0)`. You can check [Hint: you can check this page](https://keras.io/api/layers/core_layers/dense/).
2. We are training our model with sequence of `1024` length, however we are doing inference with sequence of `2000` length. What is happening here?

In [ ]:
# Build Token Classification model
backbone = keras_nlp.models.DebertaV3Backbone.from_preset(
    CFG.preset,
)
out = backbone.output
out = keras.layers.Dense(CFG.num_labels, name="logits")(out)
out = keras.layers.Activation("softmax", dtype="float32", name="prediction")(out)
model = keras.models.Model(backbone.input, out)

# Compile model for optimizer, loss and metric
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=2e-5),
    loss=CrossEntropy(),
    metrics=[FBetaScore()],
)

# Summary of the model architecture
model.summary()

# ⚓ | LR Schedule

A well-structured learning rate schedule is essential for efficient model training, ensuring optimal convergence and avoiding issues such as overshooting or stagnation.

In [ ]:
import math

def get_lr_callback(batch_size=8, mode='cos', epochs=10, plot=False):
    lr_start, lr_max, lr_min = 6e-6, 2.5e-6 * batch_size, 1e-6
    lr_ramp_ep, lr_sus_ep, lr_decay = 3, 0, 0.75

    def lrfn(epoch):  # Learning rate update function
        if epoch < lr_ramp_ep: lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start
        elif epoch < lr_ramp_ep + lr_sus_ep: lr = lr_max
        elif mode == 'exp': lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min
        elif mode == 'step': lr = lr_max * lr_decay**((epoch - lr_ramp_ep - lr_sus_ep) // 2)
        elif mode == 'cos':
            decay_total_epochs, decay_epoch_index = epochs - lr_ramp_ep - lr_sus_ep + 3, epoch - lr_ramp_ep - lr_sus_ep
            phase = math.pi * decay_epoch_index / decay_total_epochs
            lr = (lr_max - lr_min) * 0.5 * (1 + math.cos(phase)) + lr_min
        return lr

    if plot:  # Plot lr curve if plot is True
        fig = px.line(x=np.arange(epochs),
                      y=[lrfn(epoch) for epoch in np.arange(epochs)],
                      title='LR Scheduler',
                      markers=True,
                      labels={'x': 'epoch', 'y': 'lr'})
        fig.update_layout(
            yaxis = dict(
                showexponent = 'all',
                exponentformat = 'e'
            )
        )
        fig.show()

    return keras.callbacks.LearningRateScheduler(lrfn, verbose=False)  # Create lr callback

In [ ]:
lr_cb = get_lr_callback(CFG.train_batch_size, mode=CFG.lr_mode, plot=True)

# 🚂 | Training

In [ ]:
if CFG.train:
    history = model.fit(
        train_ds,
        validation_data=valid_ds,
        epochs=CFG.epochs,
        callbacks=[lr_cb],
        verbose=1,
    )
else:
    model.load_weights("/kaggle/input/pii-data-detection-ckpt-ds/model.weights.h5")

Even though this notebook does both training and inference, let's store the **weights** of the trained model on disk in case we may need it later. Also, if you need more time for inference, you can create separate notebooks for training and inference.

> **Note**: The filename of the weights should end in `.weights.h5`

In [ ]:
model.save_weights("model.weights.h5")

# 🔬 | Evaluation

We have trained and validated our model on a `1024` sequence length; however, we will be making inference using a `2000` sequence length. Thus, it is important to check how our model performs with `2000` sequence length inputs.

In [ ]:
# Build Validation dataloader with "infer_seq_len"
valid_ds = build_dataset(valid_words, valid_labels, return_ids=False, batch_size=CFG.infer_batch_size,
                        seq_len=CFG.infer_seq_len, shuffle=False, cache=False)

In [ ]:
# Evaluate
model.evaluate(valid_ds, return_dict=True, verbose=0)

# 🧪 | Prediction

## Build Test Dataloader

In [ ]:
# Test data
test_data = json.load(open(f"{BASE_PATH}/test.json"))

# Ensure number of samples is divisble by number of devices
need_samples  = len(devices) - len(test_data) % len(devices)
for _ in range(need_samples):
    test_data.append(test_data[-1]) # repeat the last sample

# Initialize empty arrays
test_words = np.empty(len(test_data), dtype=object)
test_docs = np.empty(len(test_data), dtype=np.int32)

# Fill the arrays
for i, x in tqdm(enumerate(test_data), total=len(test_data)):
    test_words[i] = np.array(x["tokens"])
    test_docs[i] = x["document"]

# Get token ids
id_ds = build_dataset(test_words, return_ids=True, batch_size=len(test_words),
                        seq_len=CFG.infer_seq_len, shuffle=False, cache=False, drop_remainder=False)
test_token_ids = ops.convert_to_numpy([ids for ids in iter(id_ds)][0])

# Build test dataloader
test_ds = build_dataset(test_words, return_ids=False, batch_size=CFG.infer_batch_size,
                        seq_len=CFG.infer_seq_len, shuffle=False, cache=False, drop_remainder=False)

## Inference

In [ ]:
# Do inference
test_preds = model.predict(test_ds, verbose=1)

# Convert probabilities to class labels via max confidence
test_preds = np.argmax(test_preds, axis=-1)

## Remove Extra Samples

We need to remove the extra samples we added to the test data to ensure the number of samples is divisible by the number of devices.

In [ ]:
test_docs = test_docs[:-need_samples]
test_token_ids = test_token_ids[:-need_samples]
test_preds = test_preds[:-need_samples]
test_words = test_words[:-need_samples]

## 🧹 | Post-Processing

The following code processes the prediction to filter out unwanted parts. Specifically, it does the following:

1. It filters out any tokens of a word that are not at the start (refer to the [🥣 | Data Processing](https://www.kaggle.com/code/awsaf49/pii-data-detection-kerasnlp-starter-notebook#%F0%9F%A5%A3-%7C-Data-Processing) section for more details).
2. It removes samples labeled as `O` (BIO format), as the submission file requires only non-`O` samples.
3. It ignores predictions for special tokens like `[CLS]`, `[SEP]`, and `[PAD]`.

> **Note**: A unique feature of following post-processing is that it uses numpy vectorized operations to filter out predictions, making it very fast and efficient.

In [ ]:
document_list = []
token_id_list = []
label_id_list = []
token_list = []

for doc, token_ids, preds, tokens in tqdm(
    zip(test_docs, test_token_ids, test_preds, test_words), total=len(test_words)
):
    # Create mask for filtering
    mask1 = np.concatenate(([True], token_ids[1:] != token_ids[:-1])) # ignore non-start tokens of a word
    mask2 = (preds != 12) # ignore `O` (BIO format) label -> 12 (integer format) label
    mask3 = (token_ids != -1)  # ignore [CLS], [SEP], and [PAD] tokens
    mask = (mask1 & mask2 & mask3) # merge filters

    # Apply filter
    token_ids = token_ids[mask]
    preds = preds[mask]

     # Store prediction if number of tokens is not zero
    if len(token_ids):
        token_list.extend(tokens[token_ids])
        document_list.extend([doc] * len(token_ids))
        token_id_list.extend(token_ids)
        label_id_list.extend(preds)

# 📩 | Submission

Let's build a dataframe from the predictions which will help us visually check if our model is predicting correctly or not. We also have to map **integer** labels to **string** BIO format labels.

In [ ]:
pred_df = pd.DataFrame(
    {
        "document": document_list,
        "token": token_id_list,
        "label_id": label_id_list,
        "token_string": token_list,
    }
)
pred_df = pred_df.rename_axis("row_id").reset_index() # add `row_id` column
pred_df["label"] = pred_df.label_id.map(CFG.id2label) # map integer label to BIO format label
pred_df.head(10)

In `submission.csv` we are excluding `token_string` and `label_id` from the columns as they are not part of submission file.

In [ ]:
sub_df = pred_df.drop(columns=["token_string", "label_id"]) # remove extra columns
sub_df.to_csv("submission.csv", index=False)

# 🔗 | Reference
* [Detect Fake Text: KerasNLP [TF/Torch/JAX][Train]](https://www.kaggle.com/code/awsaf49/detect-fake-text-kerasnlp-tf-torch-jax-train)
* [Token classification](https://huggingface.co/docs/transformers/en/tasks/token_classification)
* [transformer ner baseline [lb 0.854]](https://www.kaggle.com/code/nbroad/transformer-ner-baseline-lb-0-854)